<center><h1>Tabular Playground - September 2021</h1></center>
<center><h2>EDA & XGBoost Implementation</h2></center>
<center><h2>By Tariq Hussain</h2></center>

In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Imports

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Data preparation and processing

In [ ]:
train_fp = '../input/tabular-playground-series-sep-2021/train.csv'
train = pd.read_csv(train_fp)

test_fp = '../input/tabular-playground-series-sep-2021/test.csv'
test = pd.read_csv(test_fp)

In [ ]:
display(train)
print(train.info())

In [ ]:
print(train.isnull().values.any())
print(' ')
print(train.isnull().sum())

In [ ]:
train_filled = train.fillna(train.mean())

train_filled

In [ ]:
#train_dropped = train.dropna()

#train_dropped

In [ ]:
#train_dropped.info()

In [ ]:
test

In [ ]:
print(test.isnull().values.any())
print(' ')
print(test.isnull().sum())

In [ ]:
test_filled = test.fillna(test.mean())

test_filled

In [ ]:
#test_dropped = test.dropna()

#test_dropped

In [ ]:
#test_dropped.info()

# Exploratory Data Analysis

In [ ]:
plt.figure(figsize=(5,10))
sns.countplot(train.claim)
plt.show()

In [ ]:
train_corr = train.corr()
mask = np.triu(np.ones_like(train_corr, dtype=np.bool))

fig = plt.figure(figsize=(16,13))
sns.heatmap(train_corr, mask=mask)
plt.title('Correlation between features')

# Model prep, training and analysis

In [ ]:
X = train_filled.drop(['claim'], axis=1)
y = train_filled.claim

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0, test_size=0.5)


In [ ]:
'''
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

xgbr = XGBRegressor(seed=20)

params = { "max_depth": [2, 3, 4],
           "learning_rate": [0.03, 0.04, 0.05],
           "n_estimators": np.arange(100, 1000, 100),
           "colsample_bytree": np.arange(0.2, 0.7, 0.1),
            "colsample_bylevel": np.arange(0.2, 0.7, 0.1),
            "colsample_bynode": np.arange(0.2, 0.7, 0.1)
            }

reg = RandomizedSearchCV(estimator=xgbr,
                  param_distributions=params,
                  scoring='neg_root_mean_squared_error',
                    n_iter=50,
                  verbose=1)
'''

In [ ]:
#reg.fit(X_train, y_train)

#print("Best parameters:", reg.best_params_)

In [ ]:

from xgboost import XGBRegressor

params = {'n_estimators': 800,
         'learning_rate': 0.05,
         'max_depth': 4,
         'colsample_bytree': 0.2,
         'colsample_bynode': 0.6000000000000001,
         'colsample_bylevel': 0.5000000000000001,
         'objective': 'binary:logistic'}

model = XGBRegressor(**params)

model.fit(X_train, y_train,
         early_stopping_rounds=10,
         eval_set=[(X_val, y_val)],
          eval_metric='auc',
         verbose=False)

preds = model.predict(X_val)


In [ ]:

from sklearn.model_selection import cross_val_score, KFold

cv = KFold(n_splits=5, random_state=1, shuffle=True)

scores = -1 * cross_val_score(model, X, y, cv=cv, 
                              scoring='neg_mean_absolute_error', 
                              n_jobs=1)
print('MAE Scores: ', scores)
print('Average score: ', scores.mean())


In [ ]:

from sklearn.metrics import accuracy_score, mean_absolute_error, r2_score

mae = "Mean absolute error: {}".format(mean_absolute_error(y_val, preds))
print(mae)

r2 = "r2 score: {}".format(r2_score(y_val, preds))
print(r2)


# Preparing for submission

In [ ]:
final_output = model.predict(test_filled)

In [ ]:
final_output

In [ ]:
trimmed_output = final_output[:478960]

final_mae = "Mean absolute error: {}".format(mean_absolute_error(y_val, trimmed_output))
print(final_mae)

final_r2 = "r2 score: {}".format(r2_score(y_val, trimmed_output))
print(final_r2)


In [ ]:
sample_fp = '../input/tabular-playground-series-sep-2021/sample_solution.csv'
sample = pd.read_csv(sample_fp)

In [ ]:
sample

In [ ]:

submission = sample.copy()
submission['claim'] = final_output

submission


In [ ]:
submission.to_csv('submission.csv', index=False)